In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import xarray as xr
from pyCIAM.io import get_nearest_slrs
from shared import (
    DIR_SLR_AR6_RAW,
    PATH_SLIIDERS,
    PATH_SLR_INT,
    PATH_SLR_IS_RAW,
    open_dataset,
)

In [4]:
all_ds = []
global_ds = []
global_ssps = []


def open_and_convert(ds_path):
    out = open_dataset(ds_path)
    out["sea_level_change"] = (
        out.sea_level_change.pint.quantify().pint.to("meters").pint.dequantify()
    )
    return out


for kind in ["total", "verticallandmotion"]:
    ds = []
    this_ssps = []
    for conf in ["low", "medium"]:
        for ds_path in (DIR_SLR_AR6_RAW / "regional").glob(f"{kind}_*_{conf}_*"):
            print(f"Processing {ds_path}...")
            this_ssp = ds_path.name.split("_")[1]
            ssp_conf = f"{this_ssp}_{conf}"
            ds.append(open_and_convert(ds_path))
            this_ssps.append(ssp_conf)
            if kind == "total":
                global_ds.append(
                    open_and_convert(DIR_SLR_AR6_RAW / "global" / ds_path.name)
                )
                global_ssps.append(ssp_conf)
    all_ds.append(
        xr.concat(ds, pd.Index(this_ssps, name="scenario"), data_vars="different")
    )

# for some reason the VLM dataset has an entry for 2005 that is all 0s, while other
# datasets just don't have 2005 b/c it is the assumed basline
assert (all_ds[1].sea_level_change.sel(years=2005) == 0).all()
all_ds[1] = all_ds[1].sel(years=slice(2006, None))

global_ds = (
    xr.concat(global_ds, pd.Index(global_ssps, name="scenario"), data_vars="different")
    .squeeze(drop=True)
    .drop_vars(["lon", "lat"])
    .sea_level_change
)

# handle floating point matching errors on the quantile dimension
global_ds["quantiles"] = all_ds[0].quantiles
all_ds[1]["quantiles"] = all_ds[0].quantiles

all_ds = xr.Dataset(
    {
        "lsl_msl05": all_ds[0].sea_level_change,
        "lsl_ncc_msl05": all_ds[1].sea_level_change,
        "gsl_msl05": global_ds,
        "lon": all_ds[1].lon,
        "lat": all_ds[0].lat,
    }
)

# drop locations with NaN values in the time period we're interested in
valid = (
    all_ds[["lsl_msl05", "lsl_ncc_msl05"]]
    .sel(years=slice(2100))
    .notnull()
    .all(["scenario", "quantiles", "years"])
    .to_array("tmp")
    .all("tmp")
)
all_ds = all_ds.sel(locations=valid)

all_ds = all_ds.rename(
    {"years": "year", "quantiles": "quantile", "locations": "site_id"}
)

# we generally allow +180 but not -180
all_ds["lon"] = all_ds.lon.where(all_ds.lon != -180, 180)

# ensure no locations have missing values
assert all_ds.sel(year=slice(2100)).notnull().all().to_array().all()

Processing gs://rhg-data/impactlab-rhg/coastal/ciam_paper/data/raw/slr/ar6/regional/total_ssp126_low_confidence_values.nc...
Processing gs://rhg-data/impactlab-rhg/coastal/ciam_paper/data/raw/slr/ar6/regional/total_ssp245_low_confidence_values.nc...
Processing gs://rhg-data/impactlab-rhg/coastal/ciam_paper/data/raw/slr/ar6/regional/total_ssp585_low_confidence_values.nc...
Processing gs://rhg-data/impactlab-rhg/coastal/ciam_paper/data/raw/slr/ar6/regional/total_ssp119_medium_confidence_values.nc...
Processing gs://rhg-data/impactlab-rhg/coastal/ciam_paper/data/raw/slr/ar6/regional/total_ssp126_medium_confidence_values.nc...
Processing gs://rhg-data/impactlab-rhg/coastal/ciam_paper/data/raw/slr/ar6/regional/total_ssp245_medium_confidence_values.nc...
Processing gs://rhg-data/impactlab-rhg/coastal/ciam_paper/data/raw/slr/ar6/regional/total_ssp370_medium_confidence_values.nc...
Processing gs://rhg-data/impactlab-rhg/coastal/ciam_paper/data/raw/slr/ar6/regional/total_ssp585_medium_confidenc

## Add on the pulse from greenland and antarctica

In [5]:
sliiders = (
    xr.open_zarr(str(PATH_SLIIDERS))
    .load()
    .swap_dims(seg_adm="seg")
    .drop_duplicates("seg")
)

slrs = get_nearest_slrs(
    all_ds,
    sliiders[["seg_lon", "seg_lat"]].reset_coords(drop=True).to_dataframe(),
)

adders = (
    pd.read_parquet(PATH_SLR_IS_RAW)
    .loc[sliiders.seg.to_series().index]
    .assign(constant=0.01)
    .to_xarray()
    .to_array("sheet")
)

seg_ds = all_ds.sel(site_id=slrs.to_xarray())

In [6]:
lsl = (seg_ds.lsl_msl05 + adders).rename(scenario="tmp").stack(tmp2=["tmp", "sheet"])
lsl["scenario"] = lsl.tmp + "_" + lsl.sheet
lsl = lsl.set_index(tmp2="scenario").reset_coords(drop=True).rename(tmp2="scenario")

gsl = xr.ones_like(lsl.isel(seg=0, drop=True)) / 100

new_scens_added = xr.concat(
    (
        seg_ds[["lsl_msl05", "gsl_msl05"]],
        xr.Dataset({"lsl_msl05": lsl, "gsl_msl05": gsl}),
    ),
    dim="scenario",
)

out = (
    xr.merge((seg_ds.drop_dims("scenario"), new_scens_added))
    .swap_dims(seg="site_id")
    .drop_vars("seg")
    .drop_duplicates("site_id")
)

## Save

In [7]:
for v in out.variables:
    out[v].encoding.clear()

out.chunk({"site_id": 100}).to_zarr(str(PATH_SLR_INT), mode="w")